In [5]:
import numpy as np

In [21]:
n_control_points = 8


control_points3 = np.random.rand(n_control_points, 3) + 0.1

# control_points3 = np.array([[0, 0, 0],
#                             [1, -.1, 0],
#                             [1, 0, 2],
#                             [1, .1, 0],
#                             [2, .1, 0],
#                             [2, 1, 0],
#                             ])

control_points3 =  control_points3 @ np.array([ [500,    0,   0], 
                                                [   0, 500,   0], 
                                                [   0,    0, 200]])
M = np.array([[  0, -1],
              [  1, -4]])
              
h = np.array([[2, 4]]).T
n_curves = len(control_points3)

In [22]:
control_points = np.zeros((n_curves, 4, 3))
control_points[:, 3, :] = control_points3
control_points[:, 0, :] = np.roll(control_points3,axis=0, shift=1)

X =  np.eye(2) - np.linalg.matrix_power((M), n_curves)
for i in range(n_curves):
    result = 0
    for j in range(n_curves):
        result += np.linalg.matrix_power(M, j) @ h @ control_points3[i-j-1, :].reshape(1, 3)

    control_points[i, 1:3, :] = np.linalg.solve(X, result)

centered_cp = control_points - control_points.mean(axis = 0)

In [23]:
json_str = "const bps = ["

for cp in centered_cp:
    json_str += f"""
                [ 
                {{"x": {cp[0][0]}, "y": {cp[0][2]}, "z": {cp[0][1]} }},
                {{"x": {cp[1][0]}, "y": {cp[1][2]}, "z": {cp[1][1]} }},
                {{"x": {cp[2][0]}, "y": {cp[2][2]}, "z": {cp[2][1]} }},
                {{"x": {cp[3][0]}, "y": {cp[3][2]}, "z": {cp[3][1]} }}
                ],"""

json_str = json_str[:-1] + "\n]"
with open("bezier.js", "w") as f:
    f.write(json_str)